In [1]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
# Operating system
import os

In [2]:
vvProdutos = 'https://www.vv.com.br/pt/produto/detalhe/'
codigosVV= []
notFound = pd.DataFrame()
notFound["Produto"] = ''
notFound['Motivo'] = ''
cwd = os.getcwd()

In [3]:
query = pd.read_csv('produtos_bot_yury_vv.txt', encoding = 'latin1', sep=',')
df_query = pd.DataFrame(query)
df_query.drop('IDX', axis=1, inplace=True)

In [4]:
def buscaInfo(nav, codigo, index):
    if '-' not in codigo or '/' in codigo:
        return
    keys = []
    values = []
    nav.get(str(vvProdutos + codigo))
    nav.implicitly_wait(0.8)
    
    element = nav.find_elements(By.CLASS_NAME, 'itensEquipamentos')
    
    if not element:
        notFound.loc[len(notFound)] = [index, 'não encontrado no site']
        return
    
    for elemento in element:
        ps = elemento.find_elements(By.TAG_NAME, 'p')
        p = []
        for elemento in ps:
            p.append(elemento.text)
        equips = { str(index) : { p[0] : { 'Código OEM' : p[1], 'Equipamentos': p[2:] } } }
    
    keys.append('ID:')
    values.append(index)
    
    element = nav.find_element(By.CLASS_NAME, 'subtitulo')
    nome = element.find_element(By.TAG_NAME, 'h2').text
    nome = nome.replace('–', '-')
    
    keys.append('Nome:')
    values.append(nome)
    
    element = nav.find_element(By.CLASS_NAME, 'descricao')
    li = element.find_elements(By.TAG_NAME, 'li')    
    
    for elemento in li:
        for k in elemento.find_elements(By.TAG_NAME, 'h3'):
            keys.append(k.text)
        for v in elemento.find_elements(By.TAG_NAME, 'p'):
            texto = v.text
            texto = texto.replace('"', '\\"')
            texto = texto.replace("'", '\\"')
            values.append(texto)
    dict_infos = dict(zip(keys, values))
    element = nav.find_element(By.CLASS_NAME, 'listaGaleria')
    imgs = element.find_elements(By.TAG_NAME, 'img')
    src = []
    for img in imgs:
        src.append(img.get_attribute('src'))
    images = {str(index) : src}    
    dict_infos = str(dict_infos)
    dict_infos = dict_infos.replace("'", '"')
    images = str(images)
    images = images.replace("'", '"')
    equips = str(equips)
    equips = equips.replace("'", '"')
    with open(f'{cwd}/VV/infos.json', 'a') as teste:
        teste.write(dict_infos)
    with open(f'{cwd}/VV/images.json', 'a') as teste:
        teste.write(images)
    with open(f'{cwd}/VV/equips.json', 'a') as teste:
        teste.write(equips)

In [5]:
def findVVCod(nome):
    indice = nome.find('VV')
    nome = nome[indice:]
    nome = nome.replace(' ', '')
    nome = nome.split('/')
    cods = []
    for cod in nome:
        cod = cod.replace('VV-', '')
        cod = re.sub('[A-Z]', '', cod)
        cods.append(cod)
    return cods

In [6]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)

In [7]:
produtos = dict(zip(df_query['ET0100_PRODUTO'], df_query['ET0100_NOME']))
for ID, cod in produtos.items():
    cods = findVVCod(cod)
    for item in cods:
        if len(item) > 6:
            try:
                buscaInfo(nav, item, ID)
            except Exception as e:
                print(e)
        else:
            notFound.loc[len(notFound)] = [ID, 'Código sem sufixo']

In [ ]:
# buscaInfo(nav, '0714-01', '101297095')